# Solving The LP

In [40]:
# import packages
import gurobipy as gp
from gurobipy import GRB

### Setting Up The Data

In [41]:
# Set up the data in lists
var_names = ['turkey_delite', 'beef_boy', 'hungry_ham', 'club', 'all_meat']

# constraint matrix (LHS)
constr_coeff = [[4, 0, 0, 2, 3],
                [0, 4, 0, 2, 3],
                [0, 0, 4, 2, 3],
                [1, 1, 2, 2, 0],
                [1, 1, 1, 1, 1]]

constr_names = ['turkey', 'beef', 'ham', 'cheese', 'rolls']

constr_rhs = [384, 576, 480, 384, 300]

cost_per_ounce = [20*3/ 384, 42*3/576, 30*3/480, 18*3/384, 0.20]

# obj_coeff = [2.75 - cost, 3.50 - cost, .... ]

In [42]:
import numpy as np

In [43]:
A = np.array(constr_coeff)
c = np.array(cost_per_ounce)

In [44]:
# A @ c is incorrected
# corrected
c @ A

array([0.965625, 1.215625, 1.23125 , 1.60625 , 1.8875  ])

In [45]:
# Verify A @ c
cost_per_ounce
4*0.15625 + 0.140625 + 0.2 # math doesn't match
# incorrect, we need to go back and switch A and c
# c @ A math matches for output 0.965625

0.965625

In [46]:
cost_per_sandwhich = (c@A).tolist()
cost_per_sandwhich

[0.965625, 1.215625, 1.23125, 1.60625, 1.8875]

In [47]:
rev_per_sandwhich = [2.75, 3.50, 3.25, 4.00, 4.25]

In [48]:
# this won't work:
rev_per_sandwhich - cost_per_sandwhich

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [ ]:
# need to convert to numpy arrays then back to lists or loop it
# must decide what's easier for your equation - lists or numpy arrays
# loop:
profit_per_sandwhich = []
for s in range(len(rev_per_sandwhich)):
    profit_per_sandwhich.append(rev_per_sandwhich[s] - cost_per_sandwhich[s])

profit_per_sandwhich


[1.784375, 2.284375, 2.01875, 2.39375, 2.3625]

### Creating The Model

In [ ]:
m = gp.Model('food_truck')

# specify model sense
m.ModelSense = GRB.MAXIMIZE

In [ ]:
# Create the decision variables
dvars = []
for v in var_names:
    dvars.append(m.addVar(vtype = GRB.CONTINUOUS, name = v, lb = 0.0))

m.update()
m.display()

Maximize
  0.0
Subject To


C:\Users\zungz\AppData\Local\Temp\ipykernel_19076\3581731279.py:7: DeprecationWarning: Model.display() is deprecated
  m.display()


In [ ]:
# Add objective function
m.setObjective(gp.quicksum(profit_per_sandwhich[s]*dvars[s] for s in range((len(dvars)))))

m.update()
m.display()

Maximize
1.784375 turkey_delite + 2.284375 beef_boy + 2.01875 hungry_ham + 2.39375 club
+ 2.3625 all_meat
Subject To


C:\Users\zungz\AppData\Local\Temp\ipykernel_19076\2894661204.py:5: DeprecationWarning: Model.display() is deprecated
  m.display()


In [ ]:
# Add constraints
for i in range(len(constr_coeff)):
    m.addLConstr(gp.quicksum(constr_coeff[i][j]*dvars[j] for j in range (len(dvars))),
                 GRB.LESS_EQUAL,
                 rhs = constr_rhs[i],
                 name = constr_names[i])

m.update()
m.display()

Maximize
1.784375 turkey_delite + 2.284375 beef_boy + 2.01875 hungry_ham + 2.39375 club
+ 2.3625 all_meat
Subject To
  turkey: 4.0 turkey_delite + 2.0 club + 3.0 all_meat <= 384
  beef: 4.0 beef_boy + 2.0 club + 3.0 all_meat <= 576
  ham: 4.0 hungry_ham + 2.0 club + 3.0 all_meat <= 480
  cheese: turkey_delite + beef_boy + 2.0 hungry_ham + 2.0 club <= 384
  rolls: turkey_delite + beef_boy + hungry_ham + club + all_meat <= 300


C:\Users\zungz\AppData\Local\Temp\ipykernel_19076\4016935456.py:9: DeprecationWarning: Model.display() is deprecated
  m.display()


In [ ]:
for i in range(len(constr_coeff)):
    print(f'i is {i}: {constr_coeff[i]}')

    for j in range(len(dvars)):
        print(constr_coeff[i][j] * dvars[j])

# gp.quicksum(constr_coeff[i][j]*dvars[j] for j in range(len(dvars)))

i is 0: [4, 0, 0, 2, 3]
4.0 turkey_delite
0.0 beef_boy
0.0 hungry_ham
2.0 club
3.0 all_meat
i is 1: [0, 4, 0, 2, 3]
0.0 turkey_delite
4.0 beef_boy
0.0 hungry_ham
2.0 club
3.0 all_meat
i is 2: [0, 0, 4, 2, 3]
0.0 turkey_delite
0.0 beef_boy
4.0 hungry_ham
2.0 club
3.0 all_meat
i is 3: [1, 1, 2, 2, 0]
turkey_delite
beef_boy
2.0 hungry_ham
2.0 club
0.0 all_meat
i is 4: [1, 1, 1, 1, 1]
turkey_delite
beef_boy
hungry_ham
club
all_meat


In [50]:
# Optimize
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 9 8945HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 5 rows, 5 columns and 18 nonzeros
Model fingerprint: 0xada3b100
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 6e+02]
Presolve time: 0.01s
Presolved: 5 rows, 5 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0843750e+31   1.475000e+31   1.084375e+01      0s
       5    6.4600000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.460000000e+02


In [52]:
print(f'Optimal objective function value of ${m.ObjVal:0.2f}')
for v in m.getVars():
    print(f' {v.varName} = {v.X}')

Optimal objective function value of $646.00
 turkey_delite = 52.0
 beef_boy = 100.0
 hungry_ham = 76.0
 club = 40.0
 all_meat = 32.0


# Sensitivity Analysis

In [1]:
import sensitivity_analysis as sp